### Varying Image Capture Techniques

### Video-Based Capturing Methods

In [ ]:
import cv2
import time
import os
from datetime import datetime
import numpy as np

def calculate_sharpness(image):
    """Calculate sharpness using Laplacian variance."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray_image, cv2.CV_64F).var()

# --- Video Recording Section ---
output_folder = "video"
os.makedirs(output_folder, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
video_filename = os.path.join(output_folder, f"video_{timestamp}.mp4")

# Initialize camera with lower resolution for better FPS
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Reduced from 1920
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # Reduced from 1080

# Get ACTUAL camera FPS
requested_fps = 30
actual_fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Camera's actual FPS: {actual_fps}")

# Use frame-count based recording instead of time-based
record_duration = 5  # Seconds
target_frames = int(record_duration * actual_fps)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_filename, fourcc, actual_fps, 
                     (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
start_time = time.time()

while frame_count < target_frames:
    ret, frame = cap.read()
    if not ret:
        break

    out.write(frame)
    cv2.imshow('Video Preview', frame)
    frame_count += 1

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate actual performance
real_duration = time.time() - start_time
print(f"Requested: {record_duration}s, Actual: {real_duration:.2f}s")
print(f"Target frames: {target_frames}, Captured: {frame_count}")

cap.release()
out.release()
cv2.destroyAllWindows()

# --- Frame Extraction Section ---
video_name = os.path.splitext(os.path.basename(video_filename))[0]
frames_folder = os.path.join("frames", video_name)
os.makedirs(frames_folder, exist_ok=True)

cap = cv2.VideoCapture(video_filename)

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / fps  # Duration in seconds, can be fractional

print(f"Video FPS: {fps}, Total Frames: {total_frames}, Duration: {duration} sec")

# Open a text file to save clarity scores
clarity_file = os.path.join(frames_folder, "clarity_scores.txt")
with open(clarity_file, "w") as f:
    f.write("Second\tFrame\tSharpness Score\n")  # Write header for the clarity scores

# Loop through each second of video
for sec in range(int(duration)):  # Loop through each second
    start_frame = sec * fps
    frame_scores = []  # List to store (score, frame) tuples
    
    # Loop through each frame in the second
    for frame_num in range(start_frame, start_frame + fps):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()

        if ret:
            sharpness_score = calculate_sharpness(frame)  # Calculate sharpness score
            frame_scores.append((sharpness_score, frame, frame_num))
        else:
            print(f"Error reading frame: {frame_num}")

    # Sort frames by sharpness score in descending order
    frame_scores.sort(reverse=True, key=lambda x: x[0])

    # Pick top 5 frames with highest sharpness scores
    top_5_frames = frame_scores[:5]
    
    # Save top 5 frames and record sharpness scores in text file
    for saved_count, (score, frame, frame_num) in enumerate(top_5_frames):
        frame_filename = os.path.join(frames_folder, f"frame_{sec + 1}_{saved_count + 1}.jpg")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved: {frame_filename} with sharpness score {score:.2f}")

        # Save the sharpness score to the text file
        with open(clarity_file, "a") as f:
            f.write(f"{sec + 1}\tframe_{sec + 1}_{saved_count + 1}.jpg\t{score:.2f}\n")

cap.release()
print("Frame extraction and saving completed.")
print(f"Clarity scores saved in: {clarity_file}")




Camera's actual FPS: 30.0


2025-02-22 15:04:56.054 Python[9455:10908910] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-22 15:04:56.054 Python[9455:10908910] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Requested: 5s, Actual: 5.39s
Target frames: 150, Captured: 150
Video FPS: 30, Total Frames: 150, Duration: 5.0 sec
Saved: frames/video_2025-02-22_15-04-53/frame_1_1.jpg with sharpness score 78.81
Saved: frames/video_2025-02-22_15-04-53/frame_1_2.jpg with sharpness score 74.81
Saved: frames/video_2025-02-22_15-04-53/frame_1_3.jpg with sharpness score 73.68
Saved: frames/video_2025-02-22_15-04-53/frame_1_4.jpg with sharpness score 73.23
Saved: frames/video_2025-02-22_15-04-53/frame_1_5.jpg with sharpness score 73.12
Saved: frames/video_2025-02-22_15-04-53/frame_2_1.jpg with sharpness score 68.97
Saved: frames/video_2025-02-22_15-04-53/frame_2_2.jpg with sharpness score 68.01
Saved: frames/video_2025-02-22_15-04-53/frame_2_3.jpg with sharpness score 67.51
Saved: frames/video_2025-02-22_15-04-53/frame_2_4.jpg with sharpness score 67.38
Saved: frames/video_2025-02-22_15-04-53/frame_2_5.jpg with sharpness score 66.78
Saved: frames/video_2025-02-22_15-04-53/frame_3_1.jpg with sharpness score 

: 

In [23]:
import cv2

cap = cv2.VideoCapture(0)  # Open the webcam

# Get the current resolution of the webcam
current_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
current_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Print current resolution
print(f"Current resolution: {current_width}x{current_height}")

# Try setting different resolutions and check if the camera accepts them
supported_resolutions = [
    (640, 480),
    (1280, 720),
    (1920, 1080),
    (2560, 1440),
    (3840, 2160)
]

for width, height in supported_resolutions:
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    test_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    test_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    if (test_width == width) and (test_height == height):
        # Get the FPS for the current resolution
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(f"Supported resolution: {width}x{height} with FPS: {fps}")
    else:
        print(f"Resolution {width}x{height} not supported.")

cap.release()


Current resolution: 1920x1080
Supported resolution: 640x480 with FPS: 30.0
Supported resolution: 1280x720 with FPS: 30.0
Supported resolution: 1920x1080 with FPS: 30.0
Resolution 2560x1440 not supported.
Resolution 3840x2160 not supported.


### Seconds Based Frame Capturing

In [ ]:
import cv2
import time
import os
import numpy as np
from datetime import datetime

# Function to calculate clarity using Laplacian variance
def calculate_clarity(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray_image, cv2.CV_64F).var()

# Create folder: Images/YYYY-MM-DD_HH-MM-SS
current_timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
image_folder = os.path.join("Images", current_timestamp)
os.makedirs(image_folder, exist_ok=True)

# Open the default camera
cap = cv2.VideoCapture(0)

# Set video properties (Optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Reduced resolution for stability
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Get actual resolution and FPS
actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
actual_fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Camera resolution: {actual_width}x{actual_height}, FPS: {actual_fps}")

start_time = time.time()
record_duration = 6  # Seconds
frame_interval = 0.2  # 5 FPS

last_second = int(start_time)
frame_counter = 1

# Open the text file to save clarity scores
clarity_file = os.path.join(image_folder, "clarity_scores.txt")
with open(clarity_file, "w") as f:
    f.write("Frame\tClarity Score\n")  # Write header for the clarity scores

while time.time() - start_time < record_duration:
    elapsed_time = time.time() - start_time
    current_second = int(elapsed_time)
    
    if current_second == 0:
        continue

    if current_second != last_second:
        frame_counter = 1
        last_second = current_second

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Display preview with clarity score
    clarity_score = calculate_clarity(frame)
    cv2.putText(frame, f"Clarity: {clarity_score:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Time: {current_second}s", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Camera Preview', frame)

    if frame_counter <= 5:
        image_filename = os.path.join(image_folder, f"second_{current_second}_{frame_counter}.jpg")
        cv2.imwrite(image_filename, frame)
        
        # Print clarity score when the image is saved
        print(f"Image saved: {image_filename}, Clarity Score: {clarity_score:.2f}")
        
        # Save the clarity score to the text file
        with open(clarity_file, "a") as f:
            f.write(f"second_{current_second}_{frame_counter}.jpg\t{clarity_score:.2f}\n")
        
        frame_counter += 1

    # Calculate the time to wait until the next frame
    next_frame_time = start_time + (current_second - 1 + (frame_counter - 1) * frame_interval)
    wait_time = max(0, next_frame_time - time.time())
    time.sleep(wait_time)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f"All images saved in: {image_folder}")
print(f"Clarity scores saved in: {clarity_file}")

2025-02-22 15:14:19.037 Python[11751:11019167] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Camera resolution: 1280x720, FPS: 30.0
Image saved: Images/2025-02-22_15-14-18/second_1_1.jpg, Clarity Score: 62.35
Image saved: Images/2025-02-22_15-14-18/second_1_2.jpg, Clarity Score: 61.03
Image saved: Images/2025-02-22_15-14-18/second_1_3.jpg, Clarity Score: 60.33
Image saved: Images/2025-02-22_15-14-18/second_1_4.jpg, Clarity Score: 63.03
Image saved: Images/2025-02-22_15-14-18/second_1_5.jpg, Clarity Score: 61.01


2025-02-22 15:14:22.207 Python[11751:11019167] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-22 15:14:22.207 Python[11751:11019167] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Image saved: Images/2025-02-22_15-14-18/second_2_1.jpg, Clarity Score: 61.93
Image saved: Images/2025-02-22_15-14-18/second_2_2.jpg, Clarity Score: 58.80
Image saved: Images/2025-02-22_15-14-18/second_2_3.jpg, Clarity Score: 58.24
Image saved: Images/2025-02-22_15-14-18/second_2_4.jpg, Clarity Score: 58.99
Image saved: Images/2025-02-22_15-14-18/second_2_5.jpg, Clarity Score: 56.66
Image saved: Images/2025-02-22_15-14-18/second_3_1.jpg, Clarity Score: 67.70
Image saved: Images/2025-02-22_15-14-18/second_3_2.jpg, Clarity Score: 68.94
Image saved: Images/2025-02-22_15-14-18/second_3_3.jpg, Clarity Score: 68.38
Image saved: Images/2025-02-22_15-14-18/second_3_4.jpg, Clarity Score: 67.77
Image saved: Images/2025-02-22_15-14-18/second_3_5.jpg, Clarity Score: 68.48
Image saved: Images/2025-02-22_15-14-18/second_4_1.jpg, Clarity Score: 69.52
Image saved: Images/2025-02-22_15-14-18/second_4_2.jpg, Clarity Score: 69.37
Image saved: Images/2025-02-22_15-14-18/second_4_3.jpg, Clarity Score: 68.71

: 

### Clarity Verification for Specified Folders

In [16]:
import cv2
import os
import re

def is_blurry(image_path, threshold=10):
    """Check if an image is blurry using the Laplacian variance method."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()  # Compute variance
    return laplacian_var < threshold, laplacian_var  # Returns (is_blurry, score)

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    # Use regular expressions to extract two numbers from the filename like 'frame_1_6.jpg'
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))  # Return as a tuple (first_number, second_number)
    return 0, 0  # Default if match fails

# Paths to the two folders
folders = [
    "Images/2025-02-22_13-15-01",  # Change to your first folder
    "frames/video_2025-02-22_13-14-49"  # Change to your second folder
]

# Iterate through each folder
for folder in folders:
    if os.path.exists(folder):  # Check if the folder exists
        print(f"\n{'='*40}\nFolder: {folder}\n{'='*40}")  # Print divider for folder
        folder_blurry_images = []  # List to store blurry images for this folder
        total_score = 0  # Accumulate scores for average calculation
        total_images = 0  # Keep track of number of images processed
        
        # Check each frame (sorted sequentially by numeric part of filename)
        for filename in sorted(os.listdir(folder), key=extract_numbers):
            image_path = os.path.join(folder, filename)

            if filename.endswith(".jpg"):
                blurry, score = is_blurry(image_path)
                print(f"{filename} - Sharpness Score: {score} {'(Blurry)' if blurry else '(Clear)'}")

                total_score += score
                total_images += 1

                if blurry:
                    folder_blurry_images.append(filename)  # Store blurry image names

        if total_images > 0:  # Calculate average if any images were processed
            avg_score = total_score / total_images
            print(f"\nAverage Sharpness Score for this folder: {avg_score:.2f}")
        
        if folder_blurry_images:
            print("\nBlurry Images Detected in this folder:")
            for img in folder_blurry_images:
                print(img)
        else:
            print("\nNo blurry images detected in this folder.")

    else:
        print(f"Folder not found: {folder}")

print("\nBlurry image check completed.")



Folder: Images/2025-02-22_13-15-01
second_4_3.jpg - Sharpness Score: 9.687793179264773 (Blurry)
second_2_5.jpg - Sharpness Score: 10.589360159998765 (Clear)
second_2_4.jpg - Sharpness Score: 12.826164756572334 (Clear)
second_4_2.jpg - Sharpness Score: 10.02519999024237 (Clear)
second_4_1.jpg - Sharpness Score: 10.715888146157512 (Clear)
second_4_5.jpg - Sharpness Score: 10.36289610947322 (Clear)
second_2_3.jpg - Sharpness Score: 13.117137262679899 (Clear)
second_2_2.jpg - Sharpness Score: 13.087381328198434 (Clear)
second_4_4.jpg - Sharpness Score: 9.381072382251132 (Blurry)
second_2_1.jpg - Sharpness Score: 12.939317943989648 (Clear)
second_1_4.jpg - Sharpness Score: 12.738113748022442 (Clear)
second_5_1.jpg - Sharpness Score: 9.637951295692133 (Blurry)
second_1_5.jpg - Sharpness Score: 12.118929770253294 (Clear)
second_3_5.jpg - Sharpness Score: 8.936585631257898 (Blurry)
second_5_3.jpg - Sharpness Score: 10.066625188201272 (Clear)
second_5_2.jpg - Sharpness Score: 9.815162010561261

### Super Resolution Enhanced Model for Specified Folders (Laplacian variance on the FSRCNN-enhanced image - Before & After)

In [9]:
import cv2
import os
import re
import numpy as np

def apply_superres(image):
    """Use OpenCV's DNN super-resolution model to enhance the image."""
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    model_path = "models/FSRCNN_x4.pb"  # Replace with FSRCNN model path
    sr.readModel(model_path)
    sr.setModel("fsrcnn", 4)  # Set model type to "fsrcnn"
    result = sr.upsample(image)
    
    # Ensure the dimensions and depth of the result match the input image
    if result.shape[:2] != image.shape[:2]:
        result = cv2.resize(result, (image.shape[1], image.shape[0]))
    
    return result

def calculate_sharpness(image):
    """Calculate sharpness using Laplacian variance."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray_image, cv2.CV_64F).var()

def is_blurry(image_path, threshold=6):
    """Check if an image is blurry using Laplacian variance and super-res model."""
    image = cv2.imread(image_path)

    # Calculate sharpness for the original image
    original_sharpness = calculate_sharpness(image)
    
    # Apply the super-resolution model to the image
    enhanced_image = apply_superres(image)
    
    # Calculate sharpness for the enhanced image
    enhanced_sharpness = calculate_sharpness(enhanced_image)
    
    # Compare sharpness with the threshold to determine if the enhanced image is blurry
    blurry = enhanced_sharpness < threshold
    return blurry, original_sharpness, enhanced_sharpness, enhanced_image

def extract_numbers(filename):
    """Extract the numeric parts from the filename to sort properly."""
    match = re.match(r'frame_(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))  # Return as tuple of numbers
    return 0, 0  # Default if no match

# Paths to the two folders
folders = [
    "Images/2025-02-22_13-54-47",  # Replace with your first folder
    "frames/video_2025-02-22_13-52-44"  # Replace with your second folder
]

# Iterate through each folder
for folder in folders:
    if os.path.exists(folder):  # Check if the folder exists
        print(f"\n{'='*40}\nFolder: {folder}\n{'='*40}")  # Print divider for folder
        
        folder_blurry_images = []  # List to store blurry images for this folder
        total_score_before = 0  # Variable to accumulate the sharpness scores before upscale
        total_score_after = 0  # Variable to accumulate the sharpness scores after upscale
        num_images = 0  # Counter to track the number of images processed
        
        # Create the 'upscale' folder if it doesn't exist
        upscale_folder = os.path.join(folder, "upscale")
        if not os.path.exists(upscale_folder):
            os.makedirs(upscale_folder)

        # Check each frame (sorted sequentially by numeric part of filename)
        for filename in sorted(os.listdir(folder), key=extract_numbers):
            image_path = os.path.join(folder, filename)
            
            if filename.endswith(".jpg"):
                blurry, original_score, enhanced_score, enhanced_image = is_blurry(image_path)
                print(f"{filename} - Original Sharpness Score: {original_score:.2f}, Enhanced Sharpness Score: {enhanced_score:.2f} {'(Blurry)' if blurry else '(Clear)'}")
                
                # Save the enhanced image in the "upscale" folder
                upscale_image_path = os.path.join(upscale_folder, f"upscaled_{filename}")
                cv2.imwrite(upscale_image_path, enhanced_image)

                total_score_before += original_score  # Accumulate score for before upscale
                total_score_after += enhanced_score  # Accumulate score for after upscale
                num_images += 1  # Increment the image counter
                
                if blurry:
                    folder_blurry_images.append(filename)  # Store blurry image names

        # Print average score for the folder
        if num_images > 0:
            avg_score_before = total_score_before / num_images
            avg_score_after = total_score_after / num_images
            print(f"\nAverage Sharpness Score for {folder} (Before): {avg_score_before:.2f}")
            print(f"Average Sharpness Score for {folder} (After): {avg_score_after:.2f}")
        else:
            print(f"\nNo images to process in {folder}.")
        
        # Print the blurry images detected in this folder
        if folder_blurry_images:
            print("\nBlurry Images Detected:")
            for img in folder_blurry_images:
                print(img)
        else:
            print("\nNo blurry images detected in this folder.")
    else:
        print(f"Folder not found: {folder}")

print("\nBlurry image check completed.")



Folder: Images/2025-02-22_13-54-47
second_4_3.jpg - Original Sharpness Score: 7.88, Enhanced Sharpness Score: 7.97 (Clear)
second_2_5.jpg - Original Sharpness Score: 7.93, Enhanced Sharpness Score: 8.00 (Clear)
second_2_4.jpg - Original Sharpness Score: 8.33, Enhanced Sharpness Score: 8.41 (Clear)
second_4_2.jpg - Original Sharpness Score: 7.97, Enhanced Sharpness Score: 8.06 (Clear)
second_4_1.jpg - Original Sharpness Score: 8.21, Enhanced Sharpness Score: 8.30 (Clear)
second_4_5.jpg - Original Sharpness Score: 7.72, Enhanced Sharpness Score: 7.80 (Clear)
second_2_3.jpg - Original Sharpness Score: 8.50, Enhanced Sharpness Score: 8.57 (Clear)
second_2_2.jpg - Original Sharpness Score: 8.32, Enhanced Sharpness Score: 8.40 (Clear)
second_4_4.jpg - Original Sharpness Score: 7.80, Enhanced Sharpness Score: 7.87 (Clear)
second_2_1.jpg - Original Sharpness Score: 8.30, Enhanced Sharpness Score: 8.38 (Clear)
second_1_4.jpg - Original Sharpness Score: 7.99, Enhanced Sharpness Score: 8.09 (Cle